In [73]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 数据加载
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('Y_train.csv').squeeze()
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('Y_test.csv').squeeze()

# 确保 X_train 和 X_test 是 DataFrame
if not isinstance(X_train, pd.DataFrame):
    X_train = pd.DataFrame(X_train)
if not isinstance(X_test, pd.DataFrame):
    X_test = pd.DataFrame(X_test)

# 添加常数项
X_train = sm.add_constant(X_train, has_constant='add')
X_test = sm.add_constant(X_test, has_constant='add')

# 检查特征维度
print("X_train 形状:", X_train.shape)  # 应该是 (n_samples, 15)
print("X_test 形状:", X_test.shape)    # 应该是 (m_samples, 15)

# 设置迭代次数和分位数
iter_num = 1000
q = [0.1 * i for i in range(2,9,2)]

# 训练模型并进行预测
quan_models = [sm.QuantReg(y_train, X_train).fit(q_i, max_iter=iter_num) for q_i in q]
y_predict = [model.predict(X_test) for model in quan_models]

# 将预测结果转换为DataFrame
y_predict_df = pd.DataFrame(np.column_stack(y_predict), columns=[f'q_{q_i:.1f}' for q_i in q])

# 将预测结果保存到CSV文件
y_predict_df.to_csv('predictions.csv', index=False)

print("预测结果已保存到 predictions.csv")

X_train 形状: (1638, 15)
X_test 形状: (546, 15)


d:\Conda\envs\NNLLL\lib\site-packages\statsmodels\regression\quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


预测结果已保存到 predictions.csv


In [70]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error

mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]
ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1]/sum(mpc_bus[j][1] for j in range(118))
print(ratio_bus)
print(sum(ratio_bus[i] for i in range(118)))
print(sum(mpc_bus[j][1] for j in range(118)))

# 读取系统参数
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) # 拓扑结构

unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit', header=0) # 每个节点机组容量、电价、上/下行备用，上/下行价格

reserve_up = 150
reserve_down = 150
nodes = 118
test_Y_splits = np.array(pd.read_csv('predictions.csv', header=0))
real_Y_splits = np.array(pd.read_csv('Y_test.csv', header=0))
print(unit.shape[0])
s = test_Y_splits.shape[0]
n = len(q)
print(real_Y_splits[100][0]*ratio_bus[1])


[0.01202263 0.00471476 0.00919378 0.00919378 0.         0.01225837
 0.00447902 0.00660066 0.         0.         0.01650165 0.01107968
 0.00801509 0.00330033 0.02121641 0.00589345 0.00259312 0.01414427
 0.0106082  0.00424328 0.00330033 0.00235738 0.00165017 0.00306459
 0.         0.         0.01673739 0.00400754 0.00565771 0.
 0.01013673 0.01390853 0.00542197 0.01390853 0.00777935 0.00730787
 0.         0.         0.00636492 0.0155587  0.0087223  0.02263083
 0.00424328 0.00377181 0.01249411 0.00660066 0.00801509 0.00471476
 0.02050919 0.00400754 0.00400754 0.00424328 0.00542197 0.02663838
 0.01485149 0.01980198 0.00282885 0.00282885 0.06529939 0.01838755
 0.         0.01815182 0.         0.         0.         0.00919378
 0.00660066 0.         0.         0.0155587  0.         0.00282885
 0.00141443 0.01603017 0.01107968 0.01603017 0.01438001 0.01673739
 0.00919378 0.03064592 0.         0.01272984 0.00471476 0.00259312
 0.00565771 0.0049505  0.         0.01131542 0.         0.03842527
 0.

In [71]:
# 建立模型
model = gp.Model()
P_c = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='P_c') # 火电机组出力
R_U = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='R_U') # 火电机组向上备用容量
R_D = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='R_D') # 火电机组向下备用容量
r_U = model.addVars(unit.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='r_U') # 火电机组上调功率
r_D = model.addVars(unit.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='r_D') # 火电机组下调功率
P_lsh = model.addVars(topo.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='P_lsh') # 切负荷
delta_DA = model.addVars(topo.shape[0], s, n, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_DA')
delta_RT = model.addVars(topo.shape[0], s, n, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_RT')
ss = model.addVars(topo.shape[0], s, n, lb=0, vtype=GRB.CONTINUOUS, name='ss') 

c = 9000 #切负荷成本
c_s = 1750 

# 设置目标函数
model.setObjective(gp.quicksum(R_U[i,j]*unit.iloc[i,5] + R_D[i,j]*unit.iloc[i,6] + P_c[i,j]*unit.iloc[i,2] for i in range(unit.shape[0]) for j in range(s))
                + gp.quicksum((r_U[i,j,l]-r_D[i,j,l])*unit.iloc[i,2] +(P_lsh[i,j,l] * c) + (ss[i,j,l]*c_s) for i in range(topo.shape[0]) for j in range(s) for l in range(n))/n, GRB.MINIMIZE) 

for k in range(s):
    # 设置约束
        # 备用需求约束
    model.addConstr(gp.quicksum(R_U[i,k] for i in range(unit.shape[0])) == reserve_up)
    model.addConstr(gp.quicksum(R_D[i,k] for i in range(unit.shape[0])) == reserve_down)
        # 机组上下行备用约束
    for i in range(unit.shape[0]):
        model.addConstr(R_U[i,k] <= unit.iloc[i,3])
        model.addConstr(R_D[i,k] <= unit.iloc[i,4])
        # 日前：每个节点的功率平衡约束
    for l in range(n):
        for t in range(topo.shape[0]):
            model.addConstr(P_c[t,k]-test_Y_splits[k][l]*ratio_bus[t]
            -gp.quicksum(topo.iloc[t,j]*(delta_DA[t,k,l]-delta_DA[j,k,l]) for j in range(topo.shape[0])) >= 0)       
        model.addConstr(delta_DA[0,k,l] == 0)
        # 日前：机组容量约束
    for i in range(unit.shape[0]):
        model.addConstr(P_c[i,k] <= unit.iloc[i,1]-R_U[i,k])
        model.addConstr(P_c[i,k] >= R_D[i,k])
        # 实时：每个节点的功率平衡约束
    for l in range(n):
        for t in range(topo.shape[0]):
            model.addConstr(P_c[t,k] + r_U[t,k,l] - r_D[t,k,l] - ss[t,k,l] - test_Y_splits[k][l]*ratio_bus[t]  + P_lsh[t,k,l]
            -gp.quicksum(topo.iloc[t,j]*(delta_RT[t,k,l]-delta_RT[j,k,l]) for j in range(topo.shape[0]))==0)
        # 实时：备用容量约束
        for i in range(unit.shape[0]):
            model.addConstr(r_U[i,k,l] <= R_U[i,k])
            model.addConstr(r_D[i,k,l] <= R_D[i,k])
        model.addConstr(delta_RT[0,k,l] == 0)  
        # 添加切负荷量大于等于0的约束
        for t in range(topo.shape[0]):
            model.addConstr(P_lsh[t,k,l] >= 0)  # 确保切负荷量大于等于0
        # 线路容量约束
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i,j] < -0.1:
                    model.addConstr(topo.iloc[i,j] * (delta_DA[i,k,l] - delta_DA[j,k,l]) <= 175)
                    model.addConstr(topo.iloc[i,j] * (delta_DA[i,k,l] - delta_DA[j,k,l]) >= -175)
        # 线路容量约束
        for i in range(nodes):
            for j in range(nodes):
                if i != j and topo.iloc[i,j] < -0.1:
                    model.addConstr(topo.iloc[i,j] * (delta_RT[i,k,l] - delta_RT[j,k,l]) <= 175)
                    model.addConstr(topo.iloc[i,j] * (delta_RT[i,k,l] - delta_RT[j,k,l]) >= -175)

# model.setParam("FeasibilityTol", 1e-9) # 设置可行性容忍度
# model.setParam("OptimalityTol", 1e-9) # 设置最优性容忍度
# model.setParam("MIPGap", 1e-18) # 设置目标间隙
# model.setParam("NumericFocus", 3) # 设置更高的数值精度    
model.optimize()
# model.computeIIS()
# model.write("model1.ilp")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13400F, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4679220 rows, 1739556 columns and 11688768 nonzeros
Model fingerprint: 0x13976b1d
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+01, 2e+03]
  Bounds range     [2e+02, 2e+02]
  RHS range        [2e-01, 8e+02]
Presolve removed 2334696 rows and 399672 columns (presolve time = 5s) ...
Presolve removed 3107832 rows and 399672 columns
Presolve time: 24.66s
Presolved: 1571388 rows, 2110836 columns, 5999448 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 2.61s

Barrier statistics:
 AA' NZ     : 1.273e+07
 Factor NZ  : 6.528e+07 (roughly 2.0 GB of memory)
 Factor Ops : 6.864e+09 (less than 1 second per iteration)
 Threads    : 

In [72]:
# 建立模型
model2 = gp.Model()
r_U = model2.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='r_U') # 火电机组上调功率
r_D = model2.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='r_D') # 火电机组下调功率
P_lsh = model2.addVars(topo.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='P_lsh') # 切负荷
delta_RT = model2.addVars(topo.shape[0], s, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_RT')
ss = model2.addVars(topo.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='ss') 

c = 9000 #切负荷成本
c_s = 1750 

# 设置目标函数
model2.setObjective(gp.quicksum(R_U[i,j].x*unit.iloc[i,5] + R_D[i,j].x*unit.iloc[i,6] + P_c[i,j].x*unit.iloc[i,2] for i in range(unit.shape[0]) for j in range(s))
                + gp.quicksum((r_U[i,j]-r_D[i,j])*unit.iloc[i,2] +(P_lsh[i,j] * c) + (ss[i,j]*c_s) for i in range(topo.shape[0]) for j in range(s)), GRB.MINIMIZE) 

for k in range(s):
    # 设置约束
        # 实时：每个节点的功率平衡约束
    for t in range(topo.shape[0]):
        model2.addConstr(P_c[t,k].x + r_U[t,k] - r_D[t,k] - ss[t,k] - real_Y_splits[k][0]*ratio_bus[t] + P_lsh[t,k]
        -gp.quicksum(topo.iloc[t,j]*(delta_RT[t,k]-delta_RT[j,k]) for j in range(topo.shape[0]))==0)
        # 实时：备用容量约束
    for i in range(unit.shape[0]):
        model2.addConstr(r_U[i,k] <= R_U[i,k].x)
        model2.addConstr(r_D[i,k] <= R_D[i,k].x)
    model2.addConstr(delta_RT[0,k] == 0)  
        # 添加切负荷量大于等于0的约束
    for t in range(topo.shape[0]):
        model2.addConstr(P_lsh[t,k] >= 0)  # 确保切负荷量大于等于0
        # 线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.1:
                model2.addConstr(topo.iloc[i,j] * (delta_RT[i,k] - delta_RT[j,k]) <= 175)
                model2.addConstr(topo.iloc[i,j] * (delta_RT[i,k] - delta_RT[j,k]) >= -175)

# model.setParam("FeasibilityTol", 1e-9) # 设置可行性容忍度
# model.setParam("OptimalityTol", 1e-9) # 设置最优性容忍度
# model.setParam("MIPGap", 1e-18) # 设置目标间隙
# model.setParam("NumericFocus", 3) # 设置更高的数值精度    
model2.optimize()
# model.computeIIS()
# model.write("model1.ilp")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13400F, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 649194 rows, 322140 columns and 1493310 nonzeros
Model fingerprint: 0x7b654a91
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [4e+01, 9e+03]
  Bounds range     [2e+02, 2e+02]
  RHS range        [5e-02, 3e+02]
Presolve removed 491400 rows and 131040 columns
Presolve time: 1.91s
Presolved: 157794 rows, 287742 columns, 672126 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.12s

Barrier statistics:
 AA' NZ     : 1.261e+06
 Factor NZ  : 2.699e+06 (roughly 200 MB of memory)
 Factor Ops : 4.954e+07 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          

In [67]:
if model2.status == GRB.Status.OPTIMAL:
    # 输出目标函数值
    objective_value = model2.ObjVal
    print(f"优化完成，目标函数的最优值为: {objective_value}")

    # 遍历每个时段
    for k in range(s):  # s 是时段数
        # 计算单个时段下所有机组的出力累加
        total_power_generation = sum(P_c[i,k].X for i in range(unit.shape[0]))
        
        # 计算单个时段下所有负荷的需求累加
        total_load_demand = sum(real_Y_splits[k][0] * ratio_bus[t] for t in range(topo.shape[0]))
        
        # 计算单个时段下所有负荷的切负荷量累加
        total_load_shedding = sum(P_lsh[t,k].X for t in range(topo.shape[0]))
        
        # 输出结果
        print(f"\n时段 {k} 的统计结果:")
        print(f"所有机组的出力累加: {total_power_generation}")
        print(f"所有负荷的需求累加: {total_load_demand}")
        print(f"所有负荷的切负荷量累加: {total_load_shedding}")
else:
    print("优化未完成或未找到最优解。")

优化完成，目标函数的最优值为: 171183152.82642257

时段 0 的统计结果:
所有机组的出力累加: 1918.49054759929
所有负荷的需求累加: 2173.500000000001
所有负荷的切负荷量累加: 142.08477656502725

时段 1 的统计结果:
所有机组的出力累加: 2036.7395785618041
所有负荷的需求累加: 2095.0
所有负荷的切负荷量累加: 26.853768018233325

时段 2 的统计结果:
所有机组的出力累加: 1875.7317277193674
所有负荷的需求累加: 1846.3333339999997
所有负荷的切负荷量累加: 0.0

时段 3 的统计结果:
所有机组的出力累加: 1683.6537419180381
所有负荷的需求累加: 1577.5833339999997
所有负荷的切负荷量累加: 0.0

时段 4 的统计结果:
所有机组的出力累加: 1657.6370356982097
所有负荷的需求累加: 1446.2499999999998
所有负荷的切负荷量累加: 0.0

时段 5 的统计结果:
所有机组的出力累加: 1607.729503186153
所有负荷的需求累加: 1370.7499999999995
所有负荷的切负荷量累加: 0.0

时段 6 的统计结果:
所有机组的出力累加: 1516.9598243668356
所有负荷的需求累加: 1285.7500000000002
所有负荷的切负荷量累加: 0.0

时段 7 的统计结果:
所有机组的出力累加: 1581.199402413736
所有负荷的需求累加: 1244.8333339999997
所有负荷的切负荷量累加: 0.0

时段 8 的统计结果:
所有机组的出力累加: 1576.4024311236592
所有负荷的需求累加: 1203.2500000000002
所有负荷的切负荷量累加: 0.0

时段 9 的统计结果:
所有机组的出力累加: 1482.7760325142358
所有负荷的需求累加: 1257.7499999999998
所有负荷的切负荷量累加: 0.0

时段 10 的统计结果:
所有机组的出力累加: 1546.1182659810975
所有负荷的需求累

In [68]:
import pandas as pd

# 获取目标函数值
objective_value = model2.ObjVal

# 将目标函数值存储到 DataFrame 中
data = {'Objective Value': [objective_value]}
df = pd.DataFrame(data)

# 将 DataFrame 保存到 CSV 文件中
df.to_csv('objective_value.csv', index=False)